In [1]:
import geopandas as gpd
import pandas as pd
from pyproj import Transformer, CRS

In [2]:
mga_crs = {}
east_limit = 144
for zone in range(54, 57):
    mga_crs[zone] =CRS.from_epsg(28300 + zone)

mga_crs



{54: <Derived Projected CRS: EPSG:28354>
 Name: GDA94 / MGA zone 54
 Axis Info [cartesian]:
 - E[east]: Easting (metre)
 - N[north]: Northing (metre)
 Area of Use:
 - name: Australia - onshore and offshore between 138°E and 144°E.
 - bounds: (138.0, -48.19, 144.01, -9.08)
 Coordinate Operation:
 - name: Map Grid of Australia zone 54
 - method: Transverse Mercator
 Datum: Geocentric Datum of Australia 1994
 - Ellipsoid: GRS 1980
 - Prime Meridian: Greenwich,
 55: <Derived Projected CRS: EPSG:28355>
 Name: GDA94 / MGA zone 55
 Axis Info [cartesian]:
 - E[east]: Easting (metre)
 - N[north]: Northing (metre)
 Area of Use:
 - name: Australia - onshore and offshore between 144°E and 150°E.
 - bounds: (144.0, -50.89, 150.01, -9.23)
 Coordinate Operation:
 - name: Map Grid of Australia zone 55
 - method: Transverse Mercator
 Datum: Geocentric Datum of Australia 1994
 - Ellipsoid: GRS 1980
 - Prime Meridian: Greenwich,
 56: <Derived Projected CRS: EPSG:28356>
 Name: GDA94 / MGA zone 56
 Axis In

In [3]:
gdf = gpd.read_file('spatial_data/Sample_Points.shp')
print(gdf.crs)
zone = 56
gdf['zone'] = zone
for lon in range(156, 108, -6):
    gdf['zone'] = gdf['zone'].where(gdf['geometry'].x > lon, zone)
    zone -= 1

gdf

epsg:4326


,Id,Site_Id,POINT_X,POINT_Y,geometry,zone
0,1,EC1,151.153499,-33.675845,POINT (151.15350 -33.67585),56
1,2,EC2,151.156800,-33.671128,POINT (151.15680 -33.67113),56
2,3,EC3,151.139326,-33.672139,POINT (151.13933 -33.67214),56
3,4,EC4,151.150757,-33.659315,POINT (151.15076 -33.65932),56
4,5,EC5,151.131735,-33.677249,POINT (151.13173 -33.67725),56
...,...,...,...,...,...,...
90,92,LC4,151.137440,-33.777249,POINT (151.13744 -33.77725),56
91,93,LC5,151.151383,-33.795933,POINT (151.15138 -33.79593),56
92,94,LC6,151.148371,-33.796366,POINT (151.14837 -33.79637),56
93,95,LC6,151.147012,-33.794053,POINT (151.14701 -33.79405),56


In [4]:

gdf['easting'],gdf['northing'] = gdf['geometry'].x, gdf['geometry'].y
for zone, crs in mga_crs.items():
    transformer = Transformer.from_crs(gdf.crs, crs, always_xy=True)
    cartesian = transformer.transform(gdf['geometry'].x, gdf['geometry'].y)
    gdf['easting'] = gdf['easting'].where(gdf['zone'] != zone, cartesian[0])
    gdf['easting'] = gdf['easting'].astype('int64')
    gdf['northing'] = gdf['northing'].where(gdf['zone'] != zone, cartesian[1])
    gdf['northing'] = gdf['northing'].astype('int64')
gdf 


,Id,Site_Id,POINT_X,POINT_Y,geometry,zone,easting,northing
0,1,EC1,151.153499,-33.675845,POINT (151.15350 -33.67585),56,328822,6272253
1,2,EC2,151.156800,-33.671128,POINT (151.15680 -33.67113),56,329119,6272782
2,3,EC3,151.139326,-33.672139,POINT (151.13933 -33.67214),56,327501,6272640
3,4,EC4,151.150757,-33.659315,POINT (151.15076 -33.65932),56,328535,6274082
4,5,EC5,151.131735,-33.677249,POINT (151.13173 -33.67725),56,326807,6272061
...,...,...,...,...,...,...,...,...
90,92,LC4,151.137440,-33.777249,POINT (151.13744 -33.77725),56,327536,6260981
91,93,LC5,151.151383,-33.795933,POINT (151.15138 -33.79593),56,328865,6258932
92,94,LC6,151.148371,-33.796366,POINT (151.14837 -33.79637),56,328587,6258879
93,95,LC6,151.147012,-33.794053,POINT (151.14701 -33.79405),56,328456,6259133


In [5]:
gdf.to_csv('AFO_Field_Sites_20220317.csv')